In [ ]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from sklearn import utils
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import gensim
import io
import time
from datetime import timedelta
import progressbar

In [ ]:
csv = 'lib/dataset'
my_df = pd.read_csv(csv,index_col=0)
my_df.dropna(inplace=True)
my_df.reset_index(drop=True,inplace=True)
my_df.info()
x=my_df.text

In [ ]:
csv = 'lib/dataset ahok'
df_wiki = pd.read_csv(csv,index_col=0)
df_wiki.info()

In [ ]:
csv = 'lib/datasetfm'
df2 = pd.read_csv(csv,index_col=0)
df2.info()

In [ ]:
my_df=my_df.drop(['target'], axis=1)
df_wiki=df_wiki.drop(['target'], axis=1)
df2=df2.drop(['target'], axis=1)
my_df=my_df.append([df_wiki,df2])
my_df.dropna(inplace=True)
my_df.reset_index(drop=True,inplace=True)
my_df.info()

x = my_df.text

In [ ]:
SEED = 2000
x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, x, test_size=.02, random_state=SEED)
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED)

In [ ]:
def labelize_tweets_ug(tweets,label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(TaggedDocument(t.split(), [prefix + '_%s' % i]))
    return result

In [ ]:
all_x = pd.concat([x_train,x_validation,x_test])
all_x_w2v = labelize_tweets_ug(all_x, 'all')

In [ ]:
len(all_x_w2v)

In [ ]:
cores=multiprocessing.cpu_count()
print(cores)

In [ ]:
def get_vectors(model, corpus, size):
    vecs = np.zeros((len(corpus), size))
    n = 0
    for i in corpus.index:
        prefix = 'all_' + str(i)
        vecs[n] = model.docvecs[prefix]
        n += 1
    return vecs

# DBOW

In [ ]:
model_ug_dbow = Doc2Vec(dm=0, vector_size=100, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_dbow.build_vocab([x for x in tqdm(all_x_w2v)])

In [ ]:
%%time
for epoch in tqdm(range(30)):
    model_ug_dbow.train(utils.shuffle([x for x in all_x_w2v]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_dbow.alpha -= 0.002
    model_ug_dbow.min_alpha = model_ug_dbow.alpha

In [ ]:
model_ug_dbow.most_similar('baik')

In [ ]:
"""
train_vecs_dbow = get_vectors(model_ug_dbow, x_train, 100)
validation_vecs_dbow = get_vectors(model_ug_dbow, x_validation, 100)
clf = LogisticRegression()
clf.fit(train_vecs_dbow, y_train)
clf.score(validation_vecs_dbow, y_validation)
"""

In [ ]:
model_ug_dbow.save('lib/d2v_dbow_indo')
model_ug_dbow = Doc2Vec.load('lib/d2v_dbow_indo')
model_ug_dbow.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
print('saved')

# DMC

In [ ]:
model_ug_dmc = Doc2Vec(dm=1, dm_concat=1, vector_size=100, window=2, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_dmc.build_vocab([x for x in tqdm(all_x_w2v)])

In [ ]:
%%time
for epoch in tqdm(range(30)):
    model_ug_dmc.train(utils.shuffle([x for x in all_x_w2v]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_dmc.alpha -= 0.002
    model_ug_dmc.min_alpha = model_ug_dmc.alpha

In [ ]:
model_ug_dmc.most_similar('bagus')

In [ ]:
"""
train_vecs_dmc = get_vectors(model_ug_dmc, x_train, 100)
validation_vecs_dmc = get_vectors(model_ug_dmc, x_validation, 100)
clf = LogisticRegression()
clf.fit(train_vecs_dmc, y_train)
clf.score(validation_vecs_dmc, y_validation)
"""

In [ ]:
model_ug_dmc.save('lib/d2v_dmc_indo')
model_ug_dmc = Doc2Vec.load('lib/d2v_dmc_indo')
print('saved')

# DMM

In [ ]:
model_ug_dmm = Doc2Vec(dm=1, dm_mean=1, vector_size=100, window=4, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_dmm.build_vocab([x for x in tqdm(all_x_w2v)])

In [ ]:
%%time
for epoch in tqdm(range(30)):
    model_ug_dmm.train(utils.shuffle([x for x in all_x_w2v]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_dmm.alpha -= 0.002
    model_ug_dmm.min_alpha = model_ug_dmm.alpha

In [ ]:
model_ug_dmm.most_similar('bagus')

In [ ]:
"""
train_vecs_dmm = get_vectors(model_ug_dmm, x_train, 100)
validation_vecs_dmm = get_vectors(model_ug_dmm, x_validation, 100)
clf = LogisticRegression()
clf.fit(train_vecs_dmm, y_train)
clf.score(validation_vecs_dmm, y_validation)
"""

In [ ]:
model_ug_dmm.save('lib/d2v_dmm_indo')
model_ug_dmm = Doc2Vec.load('lib/d2v_dmm_indo')
model_ug_dmm.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
print('saved')